### 0.**準備**

In [1]:
#google driveに接続
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#インストール
!pip install transformers fugashi ipadic
!pip install demoji
!pip install neologdn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.9/600.9 kB 5.0 MB/s eta 0:00:00
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556704 sha256=f4a59dbda0927d71dfe80c8b7a6addb4dfd07a5dff341f48126d73dfe5cc1c95
  Stored in directory: /root/.cache/pip/wheels/5b/ea/e3/2f6e0860a327daba3b030853fce4483ed37468bbf1101c59c3
Successfully built ipadic
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.6/144.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for neologdn: filename=neologdn-0.5.3-cp310-cp310-linux_x86_64.whl size=223872 sha256=31a2e91a5035f835d996c325f9b29e6440034fd31919c15d7332b7e760109f4a
  Stored in directory: /root/.cache/pip/wheels/f3/06/cd/3216b1007983e3cde801f6376c641617c37f4f520f9faed644
Successfully built neologdn


In [3]:
import demoji
import neologdn
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertJapaneseTokenizer
from transformers import BertForSequenceClassification

### 1.**モデルの読み取り**

In [4]:
#事前学習済みの日本語BERTモデル
model_name = "cl-tohoku/bert-base-japanese-whole-word-masking"
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/258k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
device = "cuda"
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=36).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
#ネガポジ予測モデルの読み込み
device = "cuda"
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=36).to(device)
model = torch.load("/content/drive/MyDrive/model/model_epoch_most.pth")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-6-2f8c92ac1638>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serializati

### 2.**モデルの作動とテスト**

In [7]:
#ラベル変換表
tag_lst=[12, 14, 21, 22, 23, 24, 25, 31, 32, 33, 34, 35, 36, 37, 42, 43, 44, 45, 46, 47, 52, 53, 54, 55, 56, 57, 62, 63, 64, 65, 66, 67, 73, 74, 75, 76, 77]

In [8]:
#modelを動かして確率を返す
def modelmain(text):

  text = text

  # タブの消去
  text = text.translate(str.maketrans({'\n': '', '\t': '', '\r': '', '\u3000': ''}))

  # 絵文字の消去
  text = demoji.replace(string=text, repl='')

  # 文字の正規化
  text = neologdn.normalize(text)

  # 大文字を小文字に
  text = text.lower()

  encoding = tokenizer(text, return_tensors="pt", max_length=512, padding="max_length", truncation=True)
  encoding = {key: value.to(device) for key, value in encoding.items()}
  output = model(**encoding)

  #softmaxを使って確率に変換
  values = torch.softmax(output.logits, dim=1)

  return(values)

In [9]:
#学習モデルが正常に作動しているか確認
values = modelmain("おはよう")
print(values)

tensor([[4.4177e-05, 9.4275e-05, 8.1482e-05, 7.2692e-04, 1.0901e-03, 2.4532e-03,
         2.0106e-04, 4.5737e-05, 4.5992e-04, 1.4931e-03, 6.6913e-03, 1.0871e-03,
         2.8816e-04, 1.0235e-04, 3.3529e-04, 1.0764e-02, 4.7963e-02, 4.6407e-02,
         5.7204e-04, 2.5109e-04, 2.4628e-04, 4.5097e-03, 3.6383e-01, 6.8689e-02,
         3.8642e-03, 2.1802e-04, 5.3883e-05, 4.4589e-04, 6.3222e-02, 2.8451e-01,
         3.5361e-03, 9.2057e-04, 1.8447e-04, 5.8255e-03, 2.7219e-02, 5.0266e-02,
         1.3036e-03]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


### 3.**対象の感情予測**

In [10]:
print("～対象の現在の感情を決めてください(1が最もネガティブ,7が最もポジティブ)～")

#対象の感情の入力をユーザーに求める(不適切な入力の際、5回繰り返す)
for i in range(5):
  user_input_emo = input("1～7の任意の数字を入力してください: ")
  if(user_input_emo in ["1","2","3","4","5","6","7"]):
    user_input_emo = int(user_input_emo)
    break
  elif(i==4):
    print("正しい入力が行われませんでした。エラー")
  else:
    print("正しい入力ではありません。もう一度入力してください")

#対象へ話しかける言葉を入力
print("\n～対象に話しかけようとしている言葉を入力してください～")
user_input_moji = input("任意の文字列を入力してください: ")
print("")

#入力した文字列をモデルへ
values = modelmain(user_input_moji)

#data_lstにはタグごとの確率を格納
data_lst=[]
most=0
most_tag=0

#タグごとの確率を格納
for tag,value in zip(tag_lst,values[0]):
  data_lst.append({"tag":tag,"value":value.item()})

#対象の前の感情を元に、現在の感情の確率を比較
for data in data_lst:
  if(user_input_emo+1 > data["tag"]/10 >= user_input_emo):
    if(data["value"]>most):
      most=data["value"]
      most_tag=data["tag"] % (user_input_emo * 10)

#最も確率の高かった感情を表示
if(most_tag == user_input_emo):
  print("対象の感情に変動がないと予測されました")
  print("対象の感情は "+str(most_tag)+" になると考えられます")
elif(most_tag > user_input_emo):
  print("対象はよりポジティブになると予測されました")
  print("対象の感情は "+str(most_tag)+" になると考えられます")
elif(most_tag < user_input_emo):
  print("対象はよりネガティブになると予測されました")
  print("対象の感情は "+str(most_tag)+" になると考えられます")

～対象の現在の感情を決めてください(1が最もネガティブ,7が最もポジティブ)～
1～7の任意の数字を入力してください: 4

～対象に話しかけようとしている言葉を入力してください～
任意の文字列を入力してください: 好きなもの教えて？

対象はよりポジティブになると予測されました
対象の感情は 5 になると考えられます
